<a href="https://colab.research.google.com/github/arjasc5231/Lingometer/blob/speaker_verification/speaker_verification/network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from eval_metrics import calculate_eer
import numpy as np

In [ ]:
class CustomModel(keras.Model):
    def test_step(self, data):
        # Unpack the data. y data 사용 안함
        x, _ = data 

        """
        이하 validate의 코드를 복붙한 것이기 때문에 어색한 것이 있을 수 있음
        * num_batch같은 건 차라리 X에서 20개 단위로 끊어지도록 할 수 있겠다.
        * 화자수, 발화 수도 compile같은 단계에서 유동적으로 넣어줄 수 있겠다.
        """
        num_batch = x.shape[0]//20  # 한번의 테스트에 20개의 발화가 있을 때, 테스트의 개수 구하기
        true_score = []   # true 발화의 점수를 저장할 배열
        false_score = []  # false 발화의 점수를 저장할 배열


        def cosine_similarity(a,b):
          def dot(A,B): return (sum(a*b for a,b in zip(A,B)))
          return dot(a,b)/((dot(a,a)**0.5)*(dot(b,b)**0.5))

        # 한번의 테스트에 5/5/10으로 구성되어있다 할 때 점수 계산하기
        for i in range(num_batch):
          y_pred = self(x[i*20:i*20+20], training=False).numpy()

          enroll = y_pred[:5]
          enroll = np.mean(enroll, axis=0) # 등록 발화 5개를 평균.             ########### 여기만 tf.reduce_average로 바꾸면 되지 않을까.

          for j in range(5): true_score.append(cosine_similarity(enroll,y_pred[5+j]))
          for j in range(10): false_score.append(cosine_similarity(enroll,y_pred[10+j]))

        # calculate_eer함수의 인자로 알맞은 형태로 변환. true의 label에 1 표시해주기
        scores = np.array(true_score+false_score)
        labels = np.array([1.0]*len(true_score)+[0.0]*len(false_score))

        # eer 계산
        eer = calculate_eer(np.arange(0, 1.0, 0.001), scores, labels)
        return {"eer": eer}

In [ ]:
def naive_model(num_speaker):
    inputs = keras.Input(shape=(128, 128, 1))
    conv1 = keras.layers.Conv2D(filters=32, kernel_size=[3, 3], activation=tf.nn.relu)(inputs)
    pool1 = keras.layers.MaxPool2D(padding='SAME')(conv1)
    conv2 = keras.layers.Conv2D(filters=64, kernel_size=[3, 3], activation=tf.nn.relu)(pool1)
    pool2 = keras.layers.MaxPool2D(padding='SAME')(conv2)
    conv3 = keras.layers.Conv2D(filters=128, kernel_size=[3, 3], activation=tf.nn.relu)(pool2)
    pool3 = keras.layers.MaxPool2D(padding='SAME')(conv3)
    
    """
    trans = keras.layers.Permute((2,1,3))(pool3)
    reshape = keras.layers.Reshape((-1, 15*128))(trans) # 열 개수(freq축)*ch
    lstm = keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=True, dropout=0.3))(reshape) # (time=15,hidden=256)

    attention_score1 = keras.layers.Dense(1, activation='tanh')(lstm) # lstm(time,hidden)*W(hidden,1)=score(time,1)
    attention_score2 = keras.layers.Softmax()(attention_score1)
    attention = keras.layers.Dot(axes=(1,1))([lstm, attention_score2]) # (time=15, hidden=256) * (time,) => (hidden=256)
    flatten = keras.layers.Flatten()(attention)
    """
    
    flatten = keras.layers.Flatten()(pool3)
    fc = keras.layers.Dense(num_speaker)(flatten)
    return CustomModel(inputs=inputs, outputs=fc)

In [ ]:
def get_network(model_name, num_speaker):
    if model_name=='naive_model': return naive_model(num_speaker)